Load packages

In [ ]:
#loading packages
push!(LOAD_PATH, "src");
import Base: @kwdef
using Test
using LifeInsuranceDataModel

using BitemporalPostgres
using SearchLight
using SearchLightPostgreSQL
using TimeZones
using ToStruct
using JSON

In [ ]:
purging the data model entirely - empty the schema

In [ ]:
if (haskey(ENV, "GITPOD_REPO_ROOT"))
    run(```psql -f sqlsnippets/droptables.sql```)
else
    println("execute this in Your REPL")
    println("""
        run(```sudo -u postgres psql -f sqlsnippets/droptables.sql```)
        """)
end


Loading the data model- Create tables, constraints etc. and load Roles

loading inverses of the role tables to provide role descriptions in object creation,  for instance like in: "ref_role=cpRole["Policy Holder"]

In [ ]:
LifeInsuranceDataModel.load_model()

cpRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ContractPartnerRole)) do entry
    cpRole[entry.value] = entry.id.value
end
tiprRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemPartnerRole)) do entry
    tiprRole[entry.value] = entry.id.value
end
titrRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemRole)) do entry
    titrRole[entry.value] = entry.id.value
end

ppRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ProductPartRole)) do entry
    ppRole[entry.value] = entry.id.value
end


Or just connect to an existing model

In [ ]:
LifeInsuranceDataModel.connect()

Create a Partner

In [ ]:
p = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="Partner 1")
w = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w)
create_component!(p, pr, w)
commit_workflow!(w)

Partner1 = p.id.value

Create tariffs

In [ ]:
# create Tariffs
function create_tariff(dsc)
    t = LifeInsuranceDataModel.Tariff()
    tr = LifeInsuranceDataModel.TariffRevision(description=dsc)
    w = Workflow(
        tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
    )
    create_entity!(w)
    create_component!(t, tr, w)
    commit_workflow!(w)
    t.id.value
end


LifeRiskTariff=create_tariff("Life Risk Insurance")
TerminalIllnessTariff=create_tariff("Terminal Illness")
OccupationalDisabilityTariff=create_tariff("Occupational Disability")
ProfitParticipationTariff=create_tariff("Profit participation")

find(TariffRevision)

In [ ]:
find(Tariff,SQLWhereExpression("id=?",ProfitParticipationTariff))
find(Tariff, SQLWhereExpression("id=?", TerminalIllnessTariff))

Create Product

In [ ]:
p = Product()
pr = ProductRevision(description="Life Risk")

pp = ProductPart()
ppr = ProductPartRevision(ref_tariff=LifeRiskTariff, ref_role=ppRole["Main Coverage - Life"], description="Main Coverage - Life")

pp2 = ProductPart()
ppr2 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Lif Risk")

w0 = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w0)
create_component!(p, pr, w0)
create_subcomponent!(p, pp, ppr, w0)
create_subcomponent!(p, pp2, ppr2, w0)
commit_workflow!(w0)

LifeRiskProduct = p.id.value
println(LifeRiskProduct)

p = Product()
pr = ProductRevision(description="Life Risk - Terminal Illness")

pp = ProductPart()
ppr = ProductPartRevision(ref_tariff=LifeRiskTariff, ref_role=ppRole["Main Coverage - Life"], description="Main Coverage - Life")

pp2 = ProductPart()
ppr2 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Life Risk")

pp3 = ProductPart()
ppr3 = ProductPartRevision(ref_tariff=TerminalIllnessTariff, ref_role=ppRole["Supplementary Coverage - Terminal Illness"], description="additional cover Terminal Illness")

pp4 = ProductPart()
ppr4 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Terminal Illness")

pp5 = ProductPart()
ppr5 = ProductPartRevision(ref_tariff=OccupationalDisabilityTariff, ref_role=ppRole["Supplementary Coverage - Terminal Illness"], description="additional cover Occupational Disablity")

pp6 = ProductPart()
ppr6 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Occ.Disablity")



w0 = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w0)
create_component!(p, pr, w0)
create_subcomponent!(p, pp, ppr, w0)
create_subcomponent!(p, pp2, ppr2, w0)
create_subcomponent!(p, pp3, ppr3, w0)
create_subcomponent!(p, pp4, ppr4, w0)
create_subcomponent!(p, pp5, ppr5, w0)
create_subcomponent!(p, pp6, ppr6, w0)
commit_workflow!(w0)

LifeRiskTIODProduct = p.id.value
println(LifeRiskTIODProduct)

In [ ]:
create contract

In [30]:
    w1 = Workflow(
        tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
    )

    create_entity!(w1)
    c = Contract()
    cr = ContractRevision(description="contract creation properties")
    create_component!(c, cr, w1)

    cpr = ContractPartnerRef(ref_super=c.id)
    cprr = ContractPartnerRefRevision(ref_partner=Partner1, ref_role=cpRole["Policy Holder"], description="policiyholder ref properties")
    create_subcomponent!(c, cpr, cprr, w1)
    # pi 1
    cpi = ProductItem(ref_super=c.id)
    cpir = ProductItemRevision(position=1, ref_product=LifeRiskTIODProduct, description="from contract creation")
    create_subcomponent!(c, cpi, cpir, w1)
    # pi 1 ti 1 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=LifeRiskTariff, ref_role=titrRole["Main Coverage - Life"], description="Life Risk tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 1 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)

    # pi 1 ti 2 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=ProfitParticipationTariff, ref_role=titrRole["Profit participation"], description="Profit participation tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 2 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)

    commit_workflow!(w1)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 30 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'30' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 30 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/Se

┌ Info: INSERT INTO contractpartnerrefrevisions ( "ref_component", "ref_role", "ref_validfrom", "ref_invalidfrom", "description", "ref_partner" ) VALUES ( E'6', E'1', E'30', E'9223372036854775807', E'policiyholder ref properties', E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
NOTICE:  NEW: (6,6,1,30,9223372036854775807,,"policiyholder ref properties",1)
┌ Info: SELECT "contractpartnerrefrevisions"."id" AS "contractpartnerrefrevisions_id", "contractpartnerrefrevisions"."ref_component" AS "contractpartnerrefrevisions_ref_component", "contractpartnerrefrevisions"."ref_role" AS "contractpartnerrefrevisions_ref_role", "contractpartnerrefrevisions"."ref_validfrom" AS "contractpartnerrefrevisions_ref_validfrom", "contractpartnerrefrevisions"."ref_invalidfrom" AS "contractpartnerrefrevisions_ref_invalidfrom", "contractpartnerrefrevisions"."description" AS "contractpartnerrefrevisions_description", "contractpa

┌ Info: INSERT INTO tariffitempartnerrefs ( "ref_history", "ref_version", "ref_super" ) VALUES ( E'30', E'9223372036854775807', E'3' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "tariffitempartnerrefs"."id" AS "tariffitempartnerrefs_id", "tariffitempartnerrefs"."ref_history" AS "tariffitempartnerrefs_ref_history", "tariffitempartnerrefs"."ref_version" AS "tariffitempartnerrefs_ref_version", "tariffitempartnerrefs"."ref_super" AS "tariffitempartnerrefs_ref_super" FROM "tariffitempartnerrefs" WHERE "id" = 1 ORDER BY tariffitempartnerrefs.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


┌ Info: INSERT INTO tariffitempartnerrefrevisions ( "ref_component", "ref_role", "ref_validfrom", "ref_invalidfrom", "description", "ref_partner" ) VALUES ( E'1', E'1', E'30', E'9223372036854775807', E'partner 1 ref properties', E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
NOTICE:  NEW: (1,1,1,30,9223372036854775807,,"partner 1 ref properties",1)
┌ Info: SELECT "tariffitempartnerrefrevisions"."id" AS "tariffitempartnerrefrevisions_id", "tariffitempartnerrefrevisions"."ref_component" AS "tariffitempartnerrefrevisions_ref_component", "tariffitempartnerrefrevisions"."ref_role" AS "tariffitempartnerrefrevisions_ref_role", "tariffitempartnerrefrevisions"."ref_validfrom" AS "tariffitempartnerrefrevisions_ref_validfrom", "tariffitempartnerrefrevisions"."ref_invalidfrom" AS "tariffitempartnerrefrevisions_ref_invalidfrom", "tariffitempartnerrefrevisions"."description" AS "tariffitempartnerrefrevisions_descrip

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO tariffitems ( "ref_history", "ref_version", "ref_super" ) VALUES ( E'30', E'9223372036854775807', E'5' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "tariffitems"."id" AS "tariffitems_id", "tariffitems"."ref_history" AS "tariffitems_ref_history", "tariffitems"."ref_version" AS "tariffitems_ref_version", "tariffitems"."ref_super" AS "tariffitems_ref_super" FROM "tariffitems" WHERE "id" = 4 ORDER BY tariffitems.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO tariffitemrevisions ( "ref_component", "ref_role", "ref_validfrom", "ref_invali